In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import transformers
import torch

In [2]:
model_name_or_directory = "/home/local/vanroy/llm-finetuning/instruct-tuning/merged/falcon-7b-ft-alpaca-cleaned-dutch"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_directory)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_directory,                                         
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
streamer = TextStreamer(tokenizer)

2023-07-02 11:07:44.084976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/local/vanroy/llm-finetuning/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/local/vanroy/llm-finetuning/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def format_alpaca_sample(instruction: str, input_text: str):
    if len(input_text) >= 2:
        text = f'''Hieronder staat een instructie `Instruction` die een taak beschrijft, gecombineerd met een invoer `Input` die verdere context biedt. Schrijf een antwoord na `Response:` dat het verzoek op de juiste manier voltooit of beantwoordt.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:

'''
    else:
        text = f'''Hieronder staat een instructie `Instruction` die een taak beschrijft. Schrijf een antwoord na `Response:` dat het verzoek op de juiste manier voltooit of beantwoordt.

### Instruction:
{instruction}

### Response:
'''
    return text

In [5]:
instruction = "Welke hoort niet in het rijtje thuis?"
input_text = "hond, kat, peer, aap, paard, vis"

input_prompt = format_alpaca_sample(instruction, input_text)
inputs = tokenizer([input_prompt], return_tensors="pt")

_ = model.generate(
    input_ids=inputs["input_ids"].to(model.device),
    attention_mask=inputs["attention_mask"].to(model.device),
    streamer=streamer,
    max_new_tokens=100,
    do_sample=True,
    top_k=3,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

/home/local/vanroy/llm-finetuning/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Hieronder staat een instructie `Instruction` die een taak beschrijft, gecombineerd met een invoer `Input` die verdere context biedt. Schrijf een antwoord na `Response:` dat het verzoek op de juiste manier voltooit of beantwoordt.

### Instruction:
Welke hoort niet in het rijtje thuis?

### Input:
hond, kat, peer, aap, paard, vis

### Response:



/home/local/vanroy/llm-finetuning/.venv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


`hond` en `kat` zijn niet in het rijtje thuis. `peer` en `aap` zijn niet in het rijtje thuis. `vis` en `paard` zijn niet in het rijtje thuis.

De vraag vraagt om een verkeerde antwoord.

Het is een fout om een fysieke item in het rijtje te noemen.
<|endoftext|>
